In [5]:
#import dependancies
import json
from elasticsearch import Elasticsearch, helpers

In [6]:
#initialize Elasticsearch client
client = Elasticsearch()

In [7]:
#query the idex called "pandemics".
client.search(index='pandemics', filter_path=['hits.hits._id', 'hits.hits._type'])

<ipython-input-7-7cad668f0c0e>:2: ElasticsearchWarning: The client is unable to verify that the server is Elasticsearch due security privileges on the server side
  client.search(index='pandemics', filter_path=['hits.hits._id', 'hits.hits._type'])


AuthenticationException: AuthenticationException(401, 'security_exception', 'missing authentication credentials for REST request [/pandemics/_search?filter_path=hits.hits._id%2Chits.hits._type]')

### Get 10 articles

In [8]:
res = client.search(index="pandemics", query= {"match_all": {}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

AuthenticationException: AuthenticationException(401, 'security_exception', 'missing authentication credentials for REST request [/pandemics/_search]')

### Create a simple (match) query and extract the results from the nested JSON response.

In [21]:
question = "vaccine"
query = {"match": {"text": question}}

res = client.search(index="pandemics", query=query, size=50)
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(article_title)s %(section_title)s" % hit["_source"])

Got 76 Hits:
HIV/AIDS Vaccination
HIV/AIDS Vaccination
Science diplomacy and pandemics See also
Science diplomacy and pandemics See also
COVID-19 pandemic Vaccine
COVID-19 pandemic Vaccine
Targeted immunization strategies See also
Targeted immunization strategies See also
Virus Vaccines
Virus Vaccines
Pandemic prevention Vaccination
Pandemic prevention Vaccination
Cholera Vaccination
Cholera Vaccination
Targeted immunization strategies Background
Targeted immunization strategies Background
Science diplomacy and pandemics Ebola
Science diplomacy and pandemics Ebola
HIV/AIDS Treatment
HIV/AIDS Treatment
COVID-19 pandemic 2020
COVID-19 pandemic 2020
COVID-19 pandemic United Kingdom
COVID-19 pandemic United Kingdom
Science diplomacy and pandemics Severe acute respiratory syndrome (SARS)
Science diplomacy and pandemics Severe acute respiratory syndrome (SARS)
Swine influenza 1976 U.S.
Swine influenza 1976 U.S.
Virus Weapons
Virus Weapons
Pandemic Measles
Pandemic Measles
Superspreading even